In [7]:
import pandas as pd

# Load the CSV file
file_path = 'stanford_2023_schedule.csv'
stanford_data = pd.read_csv(file_path)


In [16]:
# Define a function to extract Stanford's moneyline odds based on their location (home or away)
def extract_stanford_odds(row):
    if row['HomeTeam'] == 'Stanford':
        return row['HomeMoneyline']
    elif row['AwayTeam'] == 'Stanford':
        return row['AwayMoneyline']
    return None

# Apply the function to get Stanford's odds for each game
stanford_data['StanfordMoneyline'] = stanford_data.apply(extract_stanford_odds, axis=1)

# Select relevant columns for output
output = stanford_data[['Id', 'HomeTeam', 'AwayTeam', 'LineProvider', 'StanfordMoneyline']]

# Print the results
output.head(5)  # Display the first 20 rows to check the output in Jupyter Notebook


,Id,HomeTeam,AwayTeam,LineProvider,StanfordMoneyline
0,401524029,Colorado,Stanford,William Hill (New Jersey),NaN
1,401524029,Colorado,Stanford,Bovada,340.0
2,401524029,Colorado,Stanford,DraftKings,340.0
3,401524061,Stanford,California,ESPN Bet,NaN
4,401524061,Stanford,California,Bovada,210.0


In [23]:
# Filter out rows where the LineProvider is either "William Hill (New Jersey)" or "ESPN Bet"
filtered_data = stanford_data[
    (stanford_data['LineProvider'] != 'William Hill (New Jersey)') &
    (stanford_data['LineProvider'] != 'ESPN Bet') &
    (stanford_data['LineProvider'] != 'Caesars Sportsbook (Colorado)') &
    (stanford_data['LineProvider'] != 'teamrankings') &
    (stanford_data['LineProvider'] != 'DraftKings')
]

# Select relevant columns for output
filtered_output = filtered_data[['HomeTeam', 'HomeScore', 'AwayTeam', 'AwayScore','LineProvider', 'OpeningSpread','StanfordMoneyline']]

# Print the filtered results
filtered_output.head(25)

,HomeTeam,HomeScore,AwayTeam,AwayScore,LineProvider,OpeningSpread,StanfordMoneyline
1,Colorado,43,Stanford,46,Bovada,-11.5,340.0
4,Stanford,15,California,27,Bovada,4.5,210.0
8,Stanford,6,Oregon,42,Bovada,27.0,1200.0
11,Stanford,20,Arizona,21,Bovada,10.0,290.0
14,Stanford,33,Washington,42,Bovada,26.5,1300.0
18,Washington State,7,Stanford,10,Bovada,-13.5,400.0
21,Stanford,23,Notre Dame,56,Bovada,24.5,1100.0
25,Hawai'i,24,Stanford,37,Bovada,8.0,-285.0
28,Oregon State,62,Stanford,17,Bovada,-20.5,850.0
31,USC,56,Stanford,10,Bovada,-30.0,1600.0


In [26]:
# Function to calculate the dollar reward based on a fixed $100 bet
def calculate_dollar_reward(row, action):
    bet_amount = 100
    if action == 'bet_spread':
        if row['HomeTeam'] == 'Stanford' and (row['HomeScore'] - row['AwayScore']) > row['OpeningSpread']:
            return bet_amount * 0.91  # Win $91 on a $100 bet
        elif row['AwayTeam'] == 'Stanford' and (row['AwayScore'] - row['HomeScore']) > -row['OpeningSpread']:
            return bet_amount * 0.91  # Win $91 on a $100 bet
        else:
            return -bet_amount  # Lose the entire bet amount

    elif action == 'bet_moneyline':
        if row['HomeTeam'] == 'Stanford' and row['HomeScore'] > row['AwayScore']:
            odds = row['StanfordMoneyline']
            if odds > 0:  # Positive odds (e.g., +150)
                return bet_amount * (odds / 100)
            else:  # Negative odds (e.g., -150)
                return bet_amount / abs(odds) * 100
        elif row['AwayTeam'] == 'Stanford' and row['AwayScore'] > row['HomeScore']:
            odds = row['StanfordMoneyline']
            if odds > 0:
                return bet_amount * (odds / 100)
            else:
                return bet_amount / abs(odds) * 100
        else:
            return -bet_amount  # Lose the entire bet amount

    elif action == 'no_bet':
        return 0

# Define the MDP framework with logging capabilities
class StanfordBettingMDP:
    def __init__(self, data):
        self.data = data
        self.action_log = []  # Log to store actions and rewards for each game
    
    def get_state(self, row):
        return {
            'location': 'home' if row['HomeTeam'] == 'Stanford' else 'away',
            'opponent_strength': row['StanfordMoneyline'],
            'spread': row['OpeningSpread']
        }
    
    def get_actions(self):
        return ['bet_spread', 'bet_moneyline', 'no_bet']
    
    def reward(self, row, action):
        return calculate_dollar_reward(row, action)
    
    def simulate_policy(self, policy):
        total_reward = 0
        for _, row in self.data.iterrows():
            state = self.get_state(row)
            action = policy(state)
            action_reward = self.reward(row, action)
            total_reward += action_reward
            
            # Log each game with action and reward
            self.action_log.append({
                'HomeTeam': row['HomeTeam'],
                'HomeScore': row['HomeScore'],
                'AwayTeam': row['AwayTeam'],
                'AwayScore': row['AwayScore'],
                'LineProvider': row['LineProvider'],
                'OpeningSpread': row['OpeningSpread'],
                'StanfordMoneyline': row['StanfordMoneyline'],
                'Action': action,
                'Reward': action_reward
            })
        return total_reward

# Example policy function
def example_policy(state):
    if state['opponent_strength'] > 300:
        return 'bet_moneyline'
    elif abs(state['spread']) > 10:
        return 'bet_spread'
    else:
        return 'no_bet'

# Instantiate the MDP, run the policy, and create a DataFrame from the action log
mdp = StanfordBettingMDP(filtered_data)
total_reward = mdp.simulate_policy(example_policy)
action_log_df = pd.DataFrame(mdp.action_log)

# Display the results
print("Total Dollar Reward:", total_reward)
print("\nDetailed Game Log with Actions and Rewards:")
action_log_df.head(25) # Show the first 25 rows as an example

Total Dollar Reward: 140.0

Detailed Game Log with Actions and Rewards:


,HomeTeam,HomeScore,AwayTeam,AwayScore,LineProvider,OpeningSpread,StanfordMoneyline,Action,Reward
0,Colorado,43,Stanford,46,Bovada,-11.5,340.0,bet_moneyline,340.0
1,Stanford,15,California,27,Bovada,4.5,210.0,no_bet,0.0
2,Stanford,6,Oregon,42,Bovada,27.0,1200.0,bet_moneyline,-100.0
3,Stanford,20,Arizona,21,Bovada,10.0,290.0,no_bet,0.0
4,Stanford,33,Washington,42,Bovada,26.5,1300.0,bet_moneyline,-100.0
5,Washington State,7,Stanford,10,Bovada,-13.5,400.0,bet_moneyline,400.0
6,Stanford,23,Notre Dame,56,Bovada,24.5,1100.0,bet_moneyline,-100.0
7,Hawai'i,24,Stanford,37,Bovada,8.0,-285.0,no_bet,0.0
8,Oregon State,62,Stanford,17,Bovada,-20.5,850.0,bet_moneyline,-100.0
9,USC,56,Stanford,10,Bovada,-30.0,1600.0,bet_moneyline,-100.0


# Value Iteration